In [1]:
import roboticstoolbox as rtb
import torch    

from utils import denorm_q, norm_q

robot = rtb.models.DH.Cobra600() #Puma560()

model = torch.load('models/cobra600_v1.pt')
model.eval()

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=12, bias=True)
    (1): Linear(in_features=12, out_features=12, bias=True)
    (2): Linear(in_features=12, out_features=12, bias=True)
    (3): Linear(in_features=12, out_features=12, bias=True)
    (4): Linear(in_features=12, out_features=3, bias=True)
  )
)

In [2]:
from torch.autograd.functional import jacobian

from ikine import ikine_pi_jacob

def model_fkine(q):
    return model(norm_q(robot, q)).detach()

def model_jacobian(q):
    return jacobian(model, norm_q(robot, q)).squeeze()

def robot_fkine(q):
    return torch.tensor(robot.fkine(q.numpy()).t, dtype=torch.float)

def robot_jacobian(q):
    return torch.tensor(robot.jacob0(q.numpy())[:3], dtype=torch.float)


q_test = denorm_q(robot, torch.rand(robot.n))
robot_jacobian(q_test), model_jacobian(q_test)

(tensor([[ 2.0561e-02, -1.6188e-01,  7.2089e-17,  0.0000e+00],
         [ 4.9127e-01,  2.2231e-01, -9.8999e-17,  0.0000e+00],
         [ 0.0000e+00,  6.1630e-33, -1.0000e+00,  0.0000e+00]]),
 tensor([[ 2.4412e-02, -4.9757e-01, -2.0016e-03,  2.1616e-03],
         [ 8.5986e-01,  6.9670e-01, -6.2503e-04, -3.8034e-03],
         [ 4.2622e-04, -9.5126e-03, -2.1202e-01, -3.1727e-04]]))

In [3]:
criterion = torch.nn.MSELoss()
q_test = denorm_q(robot, torch.rand((3, robot.n)))

real = robot_fkine(q_test)
pred = model_fkine(q_test)

error = criterion(pred, real)

real, pred, error

(tensor([[ 0.5372, -0.2628,  0.1988],
         [ 0.1253,  0.4735,  0.3035],
         [ 0.4566, -0.3594,  0.3223]]),
 tensor([[ 0.5375, -0.2669,  0.1993],
         [ 0.1255,  0.4708,  0.3009],
         [ 0.4557, -0.3651,  0.3233]]),
 tensor(7.3042e-06))

In [4]:
q_start = denorm_q(robot, torch.rand(robot.n))
p_start = robot_fkine(q_start)

q_target = denorm_q(robot, torch.rand(robot.n))
p_target = robot_fkine(q_target)

In [5]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=robot_fkine, 
                       jacob=robot_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([-5.5321,  0.3985,  0.1738,  2.2638]),
 tensor([0.7511, 0.3985, 0.1738, 2.4646]),
 tensor([0.3500, 0.4728, 0.2132]),
 tensor([0.3500, 0.4728, 0.2132]))

In [9]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=model_fkine, 
                       jacob=robot_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([-520.2115, 1224.9121,   -2.2041,    2.2638]),
 tensor([0.7511, 0.3985, 0.1738, 2.4646]),
 tensor([0.3500, 0.4728, 0.2132]),
 tensor([0.2415, 0.5415, 2.5911]))

In [7]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=robot_fkine, 
                       jacob=model_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([0.7511, 0.3985, 0.1738, 2.4455]),
 tensor([0.7511, 0.3985, 0.1738, 2.4646]),
 tensor([0.3500, 0.4728, 0.2132]),
 tensor([0.3500, 0.4728, 0.2132]))

In [8]:
q_inv = ikine_pi_jacob(q_start, p_target, eta=0.1,
                       fkine=model_fkine, 
                       jacob=model_jacobian)

p_reached = robot_fkine(q_inv)

q_inv, q_target, p_target, p_reached

(tensor([0.7602, 0.3938, 0.1714, 2.4468]),
 tensor([0.7511, 0.3985, 0.1738, 2.4646]),
 tensor([0.3500, 0.4728, 0.2132]),
 tensor([0.3468, 0.4754, 0.2156]))